In [629]:
import pandas as pd 
import numpy as np 
print("imported")

imported


Load Dataset

In [630]:
df=pd.read_csv('selling_properties.csv') # df = dataframe 


Data cleaning

In [631]:
# we need to analize Apartment,Hotel Apartment,villas and Townhouse in Dubai
# extract only these
df=df.query('type in ["Apartment","Hotel Apartment","Villa","Townhouse"] and city == "Dubai"' )
# First look at the data
print(df.head())
print(df.info())


     price price_category       type  beds  baths  \
0  1450000         Medium  Apartment     1      2   
1  6600000           High  Townhouse     5      5   
2  4825000           High  Apartment     2      3   
4  2499999         Medium  Townhouse     3      4   
5  1150000        Average  Apartment     0      1   

                                             address   furnishing  \
0                       The Bay, Business Bay, Dubai    Furnished   
1                     Brookfield, DAMAC Hills, Dubai  Unfurnished   
2     Vida Residence Downtown, Downtown Dubai, Dubai  Unfurnished   
4                            Nima, The Valley, Dubai    Furnished   
5  The Sterling East, The Sterling, Business Bay,...  Unfurnished   

  completion_status   post_date  average_rent  ... total_parking_spaces  \
0             Ready  2024-04-15             0  ...                    0   
1             Ready  2024-04-15             0  ...                    0   
2             Ready  2024-04-18        28

In [632]:
# check duplicate_rows in data 
print(f'number of duplicate rows:{df.duplicated().sum()}')


number of duplicate rows:0


In [633]:
# check null values in data in percentage
print(f'{(df.isnull().sum()/df.shape[0])*100}')


price                       0.0
price_category              0.0
type                        0.0
beds                        0.0
baths                       0.0
address                     0.0
furnishing                  0.0
completion_status           0.0
post_date                   0.0
average_rent                0.0
building_name               0.0
year_of_completion          0.0
total_parking_spaces        0.0
total_floors                0.0
total_building_area_sqft    0.0
elevators                   0.0
area_name                   0.0
city                        0.0
country                     0.0
Latitude                    0.0
Longitude                   0.0
purpose                     0.0
dtype: float64


In [634]:
# check columns_name
print(f'columns_names:\n {df.columns}')

columns_names:
 Index(['price', 'price_category', 'type', 'beds', 'baths', 'address',
       'furnishing', 'completion_status', 'post_date', 'average_rent',
       'building_name', 'year_of_completion', 'total_parking_spaces',
       'total_floors', 'total_building_area_sqft', 'elevators', 'area_name',
       'city', 'country', 'Latitude', 'Longitude', 'purpose'],
      dtype='object')


In [635]:
# rename the long column for easy reference 
df=df.rename(columns={"year_of_completion":"completion_year",
                   "total_parking_spaces":"parking_spaces",
                   "total_floors":"floors",
                   "total_building_area_sqft":"area_sqft"})
print(df.columns)

Index(['price', 'price_category', 'type', 'beds', 'baths', 'address',
       'furnishing', 'completion_status', 'post_date', 'average_rent',
       'building_name', 'completion_year', 'parking_spaces', 'floors',
       'area_sqft', 'elevators', 'area_name', 'city', 'country', 'Latitude',
       'Longitude', 'purpose'],
      dtype='object')


CHANGE THE POST_DATE FROM OBJECT TO DATETIME 

In [636]:
df['post_date']=pd.to_datetime(df["post_date"],format="%Y-%m-%d",errors="coerce")

check correlation between "area_sqft" and beds 

In [637]:
print(df[["area_sqft","beds"]].corr()) 

           area_sqft      beds
area_sqft   1.000000 -0.255911
beds       -0.255911  1.000000


**problem:** correlation is negative (-0.25) is the biggest red flag. In real estate, as "Beds" increase, "Area" 

should always increase (Positive Correlation). A negative correlation means your data is saying "More bedrooms =
 Smaller house," which is physically impossible.

In [638]:
# 0 in area_sqft cosidered missed value
# check zero in  price and area_sqft in %
print(f'number of zero in price:{((df['price']==0).sum()/df.shape[0])*100} %') # output:0.049%

print(f'number of zero in area_sqft:{((df['area_sqft']==0).sum()/df.shape[0])*100} %') # output: 37.98%

number of zero in price:0.049606689816455246 %
number of zero in area_sqft:37.814470980086455 %


Identified that 37.81% of the dataset had missing values for Property Size (0 sqft).

deleting 37.81% of data introduces "Selection Bias." However, filling it with a generic average (Mean) is also wrong because a "Apartment" is not the same size as a "Villa."

salution:

In [639]:
# ----------------------------------------------------------
# step 1: Drop 0 prices
# ----------------------------------------------------------

df = df.query("price > 0").copy()

# ---------------------------------------------------------
# STEP 2: THE FIX - Handle Units and Garbage Data
# ---------------------------------------------------------

# A. Convert  tiny values to NaN immediately
# Any property listed as < 50 sqft is a data error (like 1, 10, etc.)
df.loc[df['area_sqft'] < 50, 'area_sqft'] = np.nan

# B.  Conversion (Square Meters -> Square Feet)
# We only convert if the area is "suspiciously small" FOR THE NUMBER OF BEDS.
def normalize_units(row):
    area=row["area_sqft"]
    beds=row["beds"]
    
    if pd.isna(area):
        return np.nan

    # Rule 1: High Beds, Low Area (e.g., 3+ beds, < 2000 area) -> Likely SqM
    if (beds >= 3) and (area < 2000) :
        return area * 10.764 
        
    
    # Rule 2: Normal Beds, Very Low Area (e.g., < 400) -> Likely SqM
    # (A 400 SqFt Studio is possible, but 200 SqFt is likely 200 SqM)
    if (area < 400):
        return area * 10.764
        
    return area

df['area_sqft'] = df.apply(normalize_units, axis=1)

# C. Final Safety Check
# If after conversion, it's STILL tiny (< 300 sqft), treat it as missing.
df.loc[df['area_sqft'] < 300, 'area_sqft'] = np.nan

# ---------------------------------------------------------
# STEP 3: IMPUTATION (Fill the NaNs)
# ---------------------------------------------------------
# Fill missing values with the Median of similar properties (Type + Beds)
df['area_sqft'] = df['area_sqft'].fillna(
    df.groupby(['type', 'beds'])['area_sqft'].transform('median')
)

# Backup fill (just by Beds)
df['area_sqft'] = df['area_sqft'].fillna(
    df.groupby('beds')['area_sqft'].transform('median')
)

# Drop any rows that still failed
df = df.dropna(subset=['area_sqft'])

# ---------------------------------------------------------
# STEP 4: OUTLIER REMOVAL
# ---------------------------------------------------------
df = df.query("area_sqft > 300 and area_sqft < 50000")

print("Cleaning Complete.")
print(df[['area_sqft','beds']].corr())

Cleaning Complete.
           area_sqft      beds
area_sqft   1.000000  0.615676
beds        0.615676  1.000000


**solved:** now the correlation is postive (0.61) 

**FEATURE_ENGINEERING:**

In [640]:
# Feature Engineering: Price Per SqFt
df["price_per_sqft"]=df["price"]/df["area_sqft"]
# 4. Feature Engineering: Rental Yield (ROI)
# Note: We only calculate this where rent data exists (is not 0)
df["rental_yeild"]=df.apply(lambda row:(row["average_rent"]/row["price"])*100 if row["average_rent"]>0 else None,axis=1)

**NOTE:** WITH ZEROS ENTRY IN "area_sqft" and "price" WE CAN NOT FIND "rental_yield" AND "price_per_sqft" BECOUSE 
DIVISION WITH ZERO MATHEMATICALY  NOT POSSIBLE SO,HANDLING ZERO IS VERY IMPORTANT FOR "rental_yield" AND "price_per_sqft"

In [641]:
# 1. Remove Unrealistic Rental Yields
# Realistically, Dubai ROI is 5-10%. Anything > 20% is likely bad data.
df = df.query("rental_yeild < 20")

# 2. Check the Top 5 again (Sanity Check)
print("Top 5 Realistic ROIs:")
print(df[['address','price','rental_yeild']].sort_values(by='rental_yeild', ascending=False).head(5))

# 3. Save with the correct file extension
df.to_csv("cleaned_selling_properties.csv", index=False)
print("File saved successfully as 'cleaned_selling_properties.csv'") 

Top 5 Realistic ROIs:
                                                 address    price  \
27260  Azizi Riviera 12, Azizi Riviera, Meydan One, M...  1199999   
21112  Le Grand Chateau A, Le Grand Chateau, JVC Dist...  2450000   
17689  Emirates Gardens, JVC District 12, Jumeirah Vi...   475000   
29620            Al Thamam 06, Al Thamam, Remraam, Dubai   475000   
1758   Marina Quays West, Marina Quays, Dubai Marina,...  1500000   

       rental_yeild  
27260     19.799766  
21112     19.693020  
17689     19.610526  
29620     19.408842  
1758      19.111400  
File saved successfully as 'cleaned_selling_properties.csv'
